<a href="https://colab.research.google.com/github/ffer200395/Binance-will-list/blob/main/Binance_Will_List_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gate_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 286 kB 5.2 MB/s 


In [ ]:
# Para realizar peticiones
import requests
# Para evitar baneos
import time
# Para cargar credenciales
import yaml
# Operaciones con la API de gateio
from gate_api import ApiClient, Configuration, SpotApi, Order

# Paso 0: Pseudocódigo

In [ ]:
# Iniciar sesión en la api de gateio para spot
# Continuamente
  # Consultar el último anuncio de Binance
  # Comprobar que el último anuncio no sea el último que se registró
  # Si el anuncio es el igual hace el time sleep para evitar el baneao de la api y se va al paso 1 (de conultar el último anuncio)
  # Si es distinto se procede a comprobar si es un anuncio válido
  # Si es válido se extrae el primer par anunciado para usdt
  # Devolver el par extraido
  # Comprar el par obtenido en gateio
  # Establecer orden de venta cuando precio supere un umbral
  # Salir
# Ejemplo archivo auth.yml
"""
---
  # CLIENT DETAILS
  gateio_api: "jdsgjdijgierjguer89g897907g97999"
  gateio_secret: "dsgfhfdhgfnhh89x8900n9890xfgh97dfb8gf6n7gf6ngn75gf75c67x5965cfb"
"""

# Paso 1: Realizar consulta a la api de Binance para obtener el último anuncio

In [ ]:
def get_last_ad():
  # Realizamos la petición a la api para obtener el último anuncio publicado
  request = requests.get("https://www.binance.com/bapi/composite/v1/public/cms/article/catalog/list/query?catalogId=48&pageNo=1&pageSize=3")
  # Si se ha podido realizar la consulta correctamente
  if request.status_code==200:
    # Devolvemos el título del primer anuncio
    return request.json()['data']['articles'][0]['title']
  else:
    return None

In [ ]:
# Ejemplo
get_last_ad()

'Binance Adds LAZIO, PORTO & More Pairs on Cross Margin and Isolated Margin'

# Paso 2: Comprobamos que el anuncio sea válido

In [ ]:
def get_new_coin(title):
  # Queremos que sea un anuncio válido para spot
  if 'Will List' in title and (('Futures' or 'Margin' or 'Innovation') not in title):
    # Obtenemos las monedas que se listarán
    coins = [word[1:-1] for word in title.split(' ') if word.isupper()]
    # Devolvemos la primera encontrada si se ha encontrado algo
    return coins[0] if len(coins)>0 else None

In [ ]:
# Ejemplo
#https://www.binance.com/en/support/announcement/5071ca5162df4f5696f93df1dacd260a
#https://www.binance.com/en/support/announcement/b76eb4e952a94959afb5964c32ddbb7a
#https://www.binance.com/en/support/announcement/1b82de6f419743e2ac940566e6639b8b
get_new_coin('Binance Will List MobileCoin (MOB) and Nexo (NEXO)')

'MOB'

# Paso 3: Inicio de sesión en la API de Gateio

In [ ]:
def login_gateio(path):
  # Cargamos credenciales
  with open(path) as file:
    auth = yaml.load(file, Loader=yaml.FullLoader)
    # Inicio de sesión
    client = Configuration(key=auth['gateio_api'], secret=auth['gateio_secret'])
    # Llamamos a spot
    return SpotApi(ApiClient(client))

In [ ]:
# Ejemplo
spot_api = login_gateio('/content/auth.yml')
spot_api

In [ ]:
# Ejemplo de venta
p = spot_api.list_tickers(currency_pair='ADA_USDT')[0].last
order_sell = Order(amount=str(10), price=p,side='sell', currency_pair='ADA_USDT',time_in_force='ioc')
spot_api.create_order(order_sell)

{'account': 'spot',
 'amount': '10',
 'auto_borrow': None,
 'auto_repay': None,
 'create_time': '1666174684',
 'create_time_ms': 1666174684811,
 'currency_pair': 'ADA_USDT',
 'fee': '0',
 'fee_currency': 'USDT',
 'fill_price': '0',
 'filled_total': '0',
 'gt_discount': False,
 'gt_fee': '0',
 'iceberg': '0',
 'id': '213718282648',
 'left': '10',
 'point_fee': '0',
 'price': '0.35862',
 'rebated_fee': '0',
 'rebated_fee_currency': 'ADA',
 'side': 'sell',
 'status': 'cancelled',
 'text': 'apiv4',
 'time_in_force': 'ioc',
 'type': 'limit',
 'update_time': '1666174684',
 'update_time_ms': 1666174684811}

# Paso 4: Establecer orden de compra y venta en Gateio para la moneda extraído

In [ ]:
def make_orders(spot_api, coin, capital, pct_sell):
  # Consultamos el precio de la moneda
  tickers = spot_api.list_tickers(currency_pair=f'{coin}_USDT')
  if len(tickers) == 1:
    # Establecemos el precio de compra y el de venta
    price_buy = float(tickers[0].last)
    price_sell = price_buy+(price_buy*pct_sell)
    # Calculamos que cantidad podemos comprar
    qty = capital/price_buy
    # Establecemos la orden de compra
    order_buy = Order(amount=str(qty), price=str(price_buy), side='buy', currency_pair=f'{coin}_USDT',time_in_force='ioc')
    spot_api.create_order(order_buy)
    # Establecemos la orden de venta
    order_sell = Order(amount=str(qty), price=str(price_sell), side='sell', currency_pair=f'{coin}_USDT',time_in_force='ioc')
    spot_api.create_order(order_sell)

# Paso 5: Lo unimos todo

In [ ]:
# Capital a invertir en la operación y porcentaje de venta
capital, pct_sell = 10, 0.5
# Dejamos la sesión iniciada en Gateio
spot_api = login_gateio('/content/auth.yml')
# Inicialmente el último anuncio es el que está
last_ad = get_last_ad()
while True:
  # Buscamos un nuevo anuncio
  new_ad = get_last_ad()
  # Si el nuevo anuncio capturado es el mismo al que ya teníamos debe continuar buacando
  if last_ad == new_ad:
    time.sleep(1)
    continue
  # Si es distinto se debe comprobar que sea un anuncio válido
  else:
    new_coin = get_new_coin(new_ad)
    # Si es una moneda válida se establecen las órdenes de compra y venta en gateio
    if new_coin!=None:
      make_orders(spot_api, new_coin, capital, pct_sell)
      # Una vez puestas salimos
      break
    else:
      time.sleep(1)
      continue
      